## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,78.78,64,32,11.92,scattered clouds
1,Rocha,UY,-34.4833,-54.3333,67.60,89,100,5.28,overcast clouds
2,Castro,BR,-24.7911,-50.0119,75.43,60,85,3.74,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,76.73,74,11,16.35,few clouds
4,Vilaka,LV,57.1846,27.6722,30.78,98,100,3.49,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,78.78,64,32,11.92,scattered clouds
2,Castro,BR,-24.7911,-50.0119,75.43,60,85,3.74,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,76.73,74,11,16.35,few clouds
10,Vaini,TO,-21.2000,-175.2000,80.76,74,20,9.22,few clouds
11,Hilo,US,19.7297,-155.0900,78.53,65,40,10.36,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   193
Country                192
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Presidencia Roque Saenz Pena,AR,78.78,scattered clouds,-26.7852,-60.4388,
2,Castro,BR,75.43,overcast clouds,-24.7911,-50.0119,
3,Rikitea,PF,76.73,few clouds,-23.1203,-134.9692,
10,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
11,Hilo,US,78.53,scattered clouds,19.7297,-155.0900,
15,Butaritari,KI,81.61,overcast clouds,3.0707,172.7902,
30,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,
32,Atuona,PF,78.53,clear sky,-9.8000,-139.0333,
33,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,
36,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name']==""].index)

In [39]:
clean_hotel_df[clean_hotel_df['Hotel Name']==""]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [48]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Presidencia Roque Saenz Pena,AR,78.78,scattered clouds,-26.7852,-60.4388,Hotel Aconcagua
2,Castro,BR,75.43,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
3,Rikitea,PF,76.73,few clouds,-23.1203,-134.9692,People ThankYou
10,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Hilo,US,78.53,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Butaritari,KI,81.61,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
30,Kapaa,US,80.92,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
32,Atuona,PF,78.53,clear sky,-9.8000,-139.0333,Villa Enata
33,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,Atlantis Hotel
36,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [50]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))